In [1]:
from __future__ import print_function
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

//anaconda/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [4]:
def merge_data():
    """
    Merged transaction and block data from json files
    """
    
    # load transaction data into dataframes and combine
    transactions1 = pd.read_json('./../data/transactions.json', lines=True)
    transactions2 = pd.read_json('./../data/new_transactions.json', lines = True)
    transactions = transactions1.append(transactions2)
    
    # load block data into dataframes and combine
    blocks1 = pd.read_json('./../data/blocks.json', lines=True)
    blocks2 = pd.read_json('./../data/blocks_more.json', lines=True)
    blocks3 = pd.read_json('./../data/blocks_2.json', lines=True)
    blocks = blocks1.append([blocks2, blocks3])
    
    # new block_id column converting floats back into integers
    blocks['block_id'] = blocks['number'].apply(lambda x: int(round(x)))
    
    # drop duplicate blocks
    blocks.drop_duplicates(subset='block_id', inplace=True)
    
    # merge transaction and block data 
    merged_df = transactions.merge(blocks, left_on='block_id', right_on='block_id', 
                                   suffixes=('_t', '_b'))
    
    # write to csv file
    merged_df.to_csv('./../data/data.csv')
    
    return merged_df

In [2]:
def clean_data(df):
    
    """
    Takes the merged data from 'merge_data()' and cleans it for modeling.
    """
    # drop columns
    
    # convert dates to datetime
    df.loc[:,'time_t'] = pd.to_datetime(df.time_t, yearfirst=True)
    df.loc[:,'time_b'] = pd.to_datetime(df.time_b, yearfirst=True)
    
    # drop unnamed column
    for col in df.columns:
        if col == 'Unnamed: 0':
            df.drop('Unnamed: 0', axis=1, inplace=True)
        else:
            continue
            
    # drop columns containing predominantly null values
    df.drop(['isContractTx', 'txIndex'], axis=1, inplace=True)
        
    # convert datatypes if necessary
    
    # convert long integers to regular integers
    df['difficulty'] = df['difficulty'].apply(lambda x: int(filter(str.isdigit, x)))
    df['reward'] = df['reward'].apply(lambda x: int(filter(str.isdigit, x)))
    df['totalFee'] = df['totalFee'].apply(lambda x: int(filter(str.isdigit, x)))
    
    # convert floats back to integers
    def float_to_int(col_list):
        for col in col_list:
            df[col] = df[col].apply(lambda x: np.rint(x))
            df[col] = df[col].values.astype(int)
            
    float_to_int(['amount', 'price', 'gasLimit_b', 'gasUsed_b'])
    
    # add new columns
    
    # price set for price per unit gas in gwei
    df['price_gwei'] = df['price'] / 1000000000.0
    
    # amount of money transfered in gwei
    df['amount_gwei'] = df['amount'] / 1000000000.0
    
    # fraction of gas used gasUsed in a single traction w.r.t. entire gasUsed in the block
    df['gasShare']  = df.gasUsed_t/df.gasUsed_b
    
    # actual price paid at the end of the transaction 
    df['gweiPaid'] = df.gasUsed_t*df.price_gwei
    
    # dict of price paid and blockid
    gweiDict = df[['gweiPaid','block_id']].groupby('block_id').sum().T.to_dict()
    
    # actual price paid per block
    df['gweiPaid_b'] = df.block_id.apply(lambda b: gweiDict[b]['gweiPaid'])
    
    # fraction of gwei paid w.r.t. the entire block
    df['gweiShare']  = df.gweiPaid/df.gweiPaid_b
    
    # transactions where no gas was used, thus it costs nothing (1 if it is free)
    df['free_t'] = (df.gasUsed_t ==0).apply(int)
    
    # separate columns for day, hour, dayofweek
    df['day'] = pd.DatetimeIndex(df['time_t']).day
    df['hour'] = pd.DatetimeIndex(df['time_t']).hour
    df['dayofweek'] = pd.DatetimeIndex(df['time_t']).dayofweek
    
    # write to csv file
    df.to_csv('./../data/clean_data.csv')
    
    return df

In [6]:
merged_df = merge_data()

In [3]:
df_toclean = pd.read_csv('./../data/data.csv')

In [4]:
clean_data(df_toclean)

,hash_t,accountNonce,amount,block_id,gasLimit_t,gasUsed_t,newContract,price,time_t,type,...,price_gwei,amount_gwei,gasShare,gweiPaid,gweiPaid_b,gweiShare,free_t,day,hour,dayofweek
0,0xfc505bf23efe13154192262c3f7b99e2517b133fe5f6...,3324654,499093201921000000,4295887,50000,21000,0,21000000000,2017-09-20 20:24:15,tx,...,21.000000,4.990932e+08,0.054830,4.410000e+05,8.627577e+06,0.051115,0,20,20,2
1,0xfa0124a64e4cedc7d75dff3ee0fca13d32b2a368e0cb...,3324658,149222093810000000,4295887,20967,9416,0,21000000000,2017-09-20 20:24:15,call,...,21.000000,1.492221e+08,0.024585,1.977360e+05,8.627577e+06,0.022919,0,20,20,2
2,0xe2a23b916903133728ab3dbfac55a56651508a971e89...,3324660,99608460653500000,4295887,50000,21000,0,21000000000,2017-09-20 20:24:15,tx,...,21.000000,9.960846e+07,0.054830,4.410000e+05,8.627577e+06,0.051115,0,20,20,2
3,0xc431fcd82ad55dc1aee932fa9965b55f8515981147a0...,3324657,199004777431000000,4295887,50000,21000,0,21000000000,2017-09-20 20:24:15,tx,...,21.000000,1.990048e+08,0.054830,4.410000e+05,8.627577e+06,0.051115,0,20,20,2
4,0xb759c34cb5b5af6f94f7002ef812b8482880d1f73d10...,3324662,1000288571660000000,4295887,50000,21000,0,21000000000,2017-09-20 20:24:15,tx,...,21.000000,1.000289e+09,0.054830,4.410000e+05,8.627577e+06,0.051115,0,20,20,2
5,0xad177c193597850a4c7c95cb990233a1190bd6118d33...,3324658,149222093810000000,4295887,50000,39512,0,21000000000,2017-09-20 20:24:15,tx,...,21.000000,1.492221e+08,0.103163,8.297520e+05,8.627577e+06,0.096174,0,20,20,2
6,0xa0e7eaa972996066971a21a0c4b51c413ed77dc1dd03...,557,399349399000000000,4295887,35000,30981,0,21000000000,2017-09-20 20:24:15,tx,...,21.000000,3.993494e+08,0.080889,6.506010e+05,8.627577e+06,0.075409,0,20,20,2
7,0x6ae39472fc63e30f55d38fa09403c7fb6d49da2776fa...,3324655,304861003483000000,4295887,50000,21000,0,21000000000,2017-09-20 20:24:15,tx,...,21.000000,3.048610e+08,0.054830,4.410000e+05,8.627577e+06,0.051115,0,20,20,2
8,0x68f70813df3be71d7792dfd7551b4ef72773c4b74e8a...,3324658,149222093810000000,4295887,2300,0,0,21000000000,2017-09-20 20:24:15,call,...,21.000000,1.492221e+08,0.000000,0.000000e+00,8.627577e+06,0.000000,1,20,20,2
9,0x63425f7ad03a1c88a87e75e4b86b785e3244128c974e...,3324661,1001551304340000000,4295887,50000,39512,0,21000000000,2017-09-20 20:24:15,tx,...,21.000000,1.001551e+09,0.103163,8.297520e+05,8.627577e+06,0.096174,0,20,20,2
